Sql credentials stuff

In [ ]:
import json

sql_credentials = json.load(open('/root/thomas/sqlcredentials.json'))

Download data to use for conv pose machine training

In [ ]:
import os
import sys
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import MetaData
from sqlalchemy import Table, select, func, and_, insert, delete, update, or_
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base

In [ ]:
# from datetime import datetime
import datetime

In [ ]:
today = datetime.date.today()
yesterday = today - datetime.timedelta(days = 1)

In [ ]:
yesterday

In [ ]:
yesterday = str(yesterday)

login

In [ ]:
base_folder = '/root/data/bati/'

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(sql_credentials["user"], sql_credentials["password"],
                                         sql_credentials["host"], sql_credentials["port"],
                                         sql_credentials["database"]))


In [ ]:
session_class = sessionmaker(bind=engine)
session = session_class()

Base = automap_base()
Base.prepare(engine, reflect=True)

# tables
keypoints = Base.classes.keypoint_annotations
detections = Base.classes.fish_detections

# query
# TODO (@Thomas) need to rethink this.
today = datetime.date.today()
yesterday = today - datetime.timedelta(days = 1)

results = session.query(keypoints, detections) \
                 .filter(keypoints.fish_detection_id == detections.id) \
                 .filter(keypoints.keypoints != None) \
                 .filter(keypoints.created_at >= yesterday) \
                 .all()

print('Keypoints annoted since {}: {}'.format(yesterday, len(results)))

In [ ]:
import time
import requests

In [ ]:
# create data
data = []
for r in results:
    k = r[0]
    d = r[1]
    kps = k.keypoints
    tmp = {'left_image_url': d.left_image_url,
           'right_image_url': d.right_image_url,
           'left_keypoints': kps.get('leftCrop', None),
           'right_keypoints': kps.get('rightCrop', None),
           'site_id': d.site_id,
           'pen_id': d.pen_id,
           'fish_detection_id': k.fish_detection_id,
           'annotator': k.annotated_by_email
            }
    data.append(tmp)

# post process
download = []
for d in data:
    if d['left_keypoints']:
        # download left image
        url = d['left_image_url']
    elif d['right_keypoints']:
        # download right image
        url = d['right_image_url']

    s = url.split('at=')[1].split('/')[0]
    ts = time.mktime(datetime.datetime.strptime(s, "%Y-%m-%dT%H:%M:%S.%f000Z").timetuple())
    date = str(datetime.datetime.utcfromtimestamp(ts).date())
    folder = '{}/images/{}/{}/{}/'.format(base_folder, d['site_id'], d['pen_id'], date)
    if not os.path.isdir(folder):
        os.makedirs(folder)
    image_name = os.path.basename(url).replace('.jpg', str(int(ts)) + '.jpg')
    destination = os.path.join(folder, image_name)
    d['local_path'] = destination
    d['date'] = date

    # download image
    if os.path.isfile(destination):
        continue
    print(destination)
    f = open(destination,'wb')
    f.write(requests.get(url).content)
    f.close()
    download.append(d)
print('{} images downloaded. Saving labels now'.format(len(download)))

In [ ]:
# download = data

In [ ]:
# format to be similar than labelbox
for d in download:
    path = d['local_path']
    side = os.path.basename(path).split('_')[0]
    if side == 'left':
        lab_kp = d['left_keypoints']
    else:
        lab_kp = d['right_keypoints']

    labels = {}
    for kpp in lab_kp:
        name = kpp['keypointType']
        labels[name] = [{'geometry': {'x': kpp['xCrop'], 'y': kpp['yCrop']}}]
    d['Label'] = labels

In [ ]:
# create label files
dates = list(set([d['date'] for d in download]))
dates

In [ ]:
for dat in ['2019-05-08']:
    data_points = [d for d in download if d['date'] == dat]
    left_urls = [d['left_image_url'] for d in data_points]
    print('{}: {}'.format(dat, len(data_points)))
    json_destination = os.path.join(base_folder, 'labels/labels_{}.json'.format(dat))
    
    if os.path.isfile(json_destination):
        # file already exists, open it and append
        labels = json.load(open(json_destination))
        print('Labels {}: {}'.format(dat, len(labels)))
        ct = 0
        for lab in labels:
            if lab['left_image_url'] not in left_urls:
                ct += 1
                data_points.append(lab)
        print(ct)
    print('{}: {}'.format(dat, len(data_points)))
    # save the json
    with open(json_destination, 'w') as f:
        json.dump(data_points, f)

In [ ]:
import numpy as np

In [ ]:
np.setdiff1d([d['fish_detection_id'] for d in data_points],
             [l['fish_detection_id'] for l in labels],
             )


In [ ]:
len(list(set([l['fish_detection_id'] for l in labels])))

In [ ]:
len(list(set()))